In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk.classify.util as util
import itertools
import pickle
from nltk.probability import FreqDist
import os.path
from statistics import mode
from nltk.classify import ClassifierI
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.probability import FreqDist
import numpy as np
import re
import math
import pandas as pd
import csv
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, Dense, Dropout, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\babyj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\babyj\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\babyj\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
emoji_dict = {}
with open('emoji.txt', 'r', encoding='latin-1') as emoji_file:
    for line in emoji_file:
        line = line.strip()
        if line:
            emoji, value = line.split('\t')
            emoji_dict[emoji] = int(value)

In [3]:
def replace_emojis(text, emoji_dict):
    for emoji, value in emoji_dict.items():
        if value == 1:
            text = re.sub(re.escape(emoji), 'happy', text)
        elif value == -1:
            text = re.sub(re.escape(emoji), 'sad', text)
    return text

In [4]:
#vincent
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
    
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove special characters
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)

    # Tokenize the words
    words = word_tokenize(text)
    
    # Apply stemming
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]

    # Apply lemmatization
    lemmatizer = WordNetLemmatizer()
    tagged = pos_tag(words)
    words = [lemmatizer.lemmatize(word, pos=get_wordnet_pos(pos)) if get_wordnet_pos(pos) else word for word, pos in tagged]

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words]
    
    return ' '.join(filtered_words)

hatefulCorpus=[]
nHatefulCorpus=[]
neutralCorpus=[]
with open("firstIter.csv", "r", encoding="utf-8") as file:
    reader = csv.DictReader(file)
    for row in reader:
        if not all(value == "" for value in row.values()):
            text = row["Comment Text"]
            newText = text.strip()
            newText = replace_emojis(newText, emoji_dict)
            newText = preprocess_text(newText)
            label = row["Label"]
            if label=='Neutral':
                neutralCorpus.append(newText)
            elif label=='Hateful':
                hatefulCorpus.append(newText)
            else:
                nHatefulCorpus.append(newText)

In [5]:
all_texts = hatefulCorpus + nHatefulCorpus + neutralCorpus
# Create frequency distribution
word_freq = FreqDist()

for text in all_texts:
    words = text.split()
    word_freq.update(words)
known_words = {word for word, freq in word_freq.items() if freq > 3}
len(known_words)

1547

In [6]:
total_words = 0
num_comments = len(all_texts)

for comment in all_texts:
    total_words += len(comment.split())

average_words = total_words / num_comments

variance = 0
for comment in all_texts:
    num_words = len(comment.split())
    variance += (num_words - average_words) ** 2

std_deviation = math.sqrt(variance / num_comments)
M = round(average_words + std_deviation)

def comment_to_vector(comment, known_words, M):
    words = comment.split()
    vector = []
    for i in range(M):
        if i < len(words):
            word = words[i]
            if word in known_words:
                vector.append(known_words.index(word) + 1)
            else:
                vector.append(0)
        else:
            vector.append(0)
    return vector

# Convert preprocessed comments to vector representation
hateful_comment_vectors = []
for comment in hatefulCorpus:
    vector = comment_to_vector(comment, list(known_words), M)
    hateful_comment_vectors.append(vector)
    

nHateful_comment_vectors = []
for comment in nHatefulCorpus:
    vector = comment_to_vector(comment, list(known_words), M)
    nHateful_comment_vectors.append(vector)
    
neutral_comment_vectors = []
for comment in neutralCorpus:
    vector = comment_to_vector(comment, list(known_words), M)
    neutral_comment_vectors.append(vector)

In [7]:
hateful_comment_vectors = np.array(hateful_comment_vectors)
nHateful_comment_vectors = np.array(nHateful_comment_vectors)
neutral_comment_vectors = np.array(neutral_comment_vectors)

all_comment_vectors = np.concatenate((hateful_comment_vectors, nHateful_comment_vectors, neutral_comment_vectors))

hateful_labels = np.ones(len(hatefulCorpus))
nHateful_labels = np.zeros(len(nHatefulCorpus))
neutral_labels = np.full(len(neutralCorpus), 2)

all_labels = np.concatenate((hateful_labels, nHateful_labels, neutral_labels))

X_train, X_test, y_train, y_test = train_test_split(all_comment_vectors, all_labels, test_size=0.2, random_state=42)

embedding_dim = 300


model = Sequential()
model.add(Embedding(input_dim=len(known_words) + 1, output_dim=embedding_dim, input_length=M))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(Dropout(0.5)) # Dropout layer for regularization
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(Dropout(0.5)) # Dropout layer for regularization
model.add(Dense(units=50, activation='relu'))
model.add(Dropout(0.5)) # Dropout layer for regularization
model.add(Flatten())
model.add(Dense(units=1, activation='tanh'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3)

model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/20
63/63 [==============================] - 3s 26ms/step - loss: 0.6340 - accuracy: 0.7398 - val_loss: 0.5316 - val_accuracy: 0.8091
Epoch 2/20
63/63 [==============================] - 1s 22ms/step - loss: 0.5714 - accuracy: 0.7801 - val_loss: 0.5448 - val_accuracy: 0.8091
Epoch 3/20
63/63 [==============================] - 1s 22ms/step - loss: 0.5524 - accuracy: 0.7781 - val_loss: 0.5153 - val_accuracy: 0.8091
Epoch 4/20
63/63 [==============================] - 1s 24ms/step - loss: 0.5069 - accuracy: 0.7761 - val_loss: 0.5155 - val_accuracy: 0.8072
Epoch 5/20
63/63 [==============================] - 1s 22ms/step - loss: 0.4536 - accuracy: 0.7915 - val_loss: 0.6090 - val_accuracy: 0.8032
Epoch 6/20
63/63 [==============================] - 1s 22ms/step - loss: 0.3731 - accuracy: 0.8179 - val_loss: 1.0394 - val_accuracy: 0.7376
